In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
import pandas as pd
import numpy as np
import itertools
from copy import copy
from pprint import pprint
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import abc
import sys
import re
sys.path.append("../")
import gpusims
import gpusims.plot.metrics as metric
from gpusims.plot.data import PlotData
from gpusims.config import Config, parse_configs
from gpusims.bench import parse_benchmarks

In [3]:
pd.set_option('display.max_rows', 700)
pd.set_option('display.max_columns', 700)

In [4]:
root_dir = Path("/Users/roman/dev/gpgpusims")
benchmark_dir = root_dir / "benchmarks"
run_dir = root_dir / "run"
assert benchmark_dir.is_dir()
assert run_dir.is_dir()

In [5]:
configs = parse_configs(benchmark_dir / "configs" / "configs.yml")
benchmarks = parse_benchmarks(benchmark_dir / "benchmarks.yml")

pprint(configs)
pprint(benchmarks)
pprint(gpusims.SIMULATORS)

{'sm6_gtx1080': Config(key='sm6_gtx1080', name='GTX 1080', path=PosixPath('/Users/roman/dev/gpgpusims/benchmarks/configs/SM6_GTX1080'), spec={'sm_count': 20, 'clock_speed': 1607}),
 'sm86_a4000': Config(key='sm86_a4000', name='A4000', path=PosixPath('/Users/roman/dev/gpgpusims/benchmarks/configs/SM86_A4000'), spec={'sm_count': 48, 'clock_speed': 735}),
 'sm86_rtx3070': Config(key='sm86_rtx3070', name='RTX 3070', path=PosixPath('/Users/roman/dev/gpgpusims/benchmarks/configs/SM86_RTX3070'), spec={'sm_count': 46, 'clock_speed': 1132})}
{'babelstream': Benchmark(/Users/roman/dev/gpgpusims/benchmarks/BabelStream),
 'cuda10-matrixmul': Benchmark(/Users/roman/dev/gpgpusims/benchmarks/CUDA10-matrixMul),
 'cuda10-transpose': Benchmark(/Users/roman/dev/gpgpusims/benchmarks/CUDA10-transpose),
 'cuda4-matrixmul': Benchmark(/Users/roman/dev/gpgpusims/benchmarks/CUDA4-matrixMul),
 'cuda6-transpose': Benchmark(/Users/roman/dev/gpgpusims/benchmarks/CUDA6-transpose),
 'vectoradd': Benchmark(/Users/roma

In [6]:
selected_simulators = copy(gpusims.SIMULATORS)
selected_configs = copy(configs)
selected_benchmarks = copy(benchmarks)

# for testing
if True:
    testing_simulators = None
    testing_configs = None
    testing_benchmarks = None
    # testing_simulators = list(gpusims.SIMULATORS.keys())
    # testing_simulators = [gpusims.NATIVE]
    # testing_simulators = [gpusims.MULTI2SIM]
    testing_configs = ["sm6_gtx1080", "sm86_a4000"]
    # testing_benchmarks = ["babelstream"] # "cuda6-transpose"]
    
    if testing_simulators is not None:
        selected_simulators = {k: v for k, v in gpusims.SIMULATORS.items() if k in testing_simulators}
    if testing_configs is not None:
        selected_configs = {k: v for k, v in configs.items() if k in testing_configs}
    if testing_benchmarks is not None:
        selected_benchmarks = {k: v for k, v in benchmarks.items() if k in testing_benchmarks}

pprint(selected_simulators)
pprint(selected_configs)
pprint(selected_benchmarks)

{'accelsim-ptx': <class 'gpusims.accelsim.AccelSimPTXBenchmarkConfig'>,
 'accelsim-sass': <class 'gpusims.accelsim_sass.AccelSimSASSBenchmarkConfig'>,
 'm2s': <class 'gpusims.multi2sim.Multi2SimBenchmarkConfig'>,
 'macsim': <class 'gpusims.macsim.MacSimBenchmarkConfig'>,
 'native': <class 'gpusims.native.NativeBenchmarkConfig'>,
 'tejas': <class 'gpusims.tejas.TejasBenchmarkConfig'>}
{'sm6_gtx1080': Config(key='sm6_gtx1080', name='GTX 1080', path=PosixPath('/Users/roman/dev/gpgpusims/benchmarks/configs/SM6_GTX1080'), spec={'sm_count': 20, 'clock_speed': 1607}),
 'sm86_a4000': Config(key='sm86_a4000', name='A4000', path=PosixPath('/Users/roman/dev/gpgpusims/benchmarks/configs/SM86_A4000'), spec={'sm_count': 48, 'clock_speed': 735})}
{'babelstream': Benchmark(/Users/roman/dev/gpgpusims/benchmarks/BabelStream),
 'cuda10-matrixmul': Benchmark(/Users/roman/dev/gpgpusims/benchmarks/CUDA10-matrixMul),
 'cuda10-transpose': Benchmark(/Users/roman/dev/gpgpusims/benchmarks/CUDA10-transpose),
 'cu

In [7]:
colors = ["#5F34FA", "#49DFE3", "#8CFA5D", "#E3BC49", "#FF7357", "#EE34FA"]
sim_color = {
    "GPUTejas": "#7E61FA", # colors[0],
    "AccelSim PTX": colors[1],
    "AccelSim SASS": colors[2],
    "Hardware": "#FF3C1E", # colors[3],
    "Multi2Sim": colors[4],
    "MacSim": colors[5],
}
abbr = {
    "GPUTejas": "Tejas",
    "AccelSim PTX": "Accel PTX",
    "AccelSim SASS": "Accel SASS",
    "Hardware": "HW",
    "Multi2Sim": "M2S",
    "MacSim": "MS",
}
margin = 50
default_layout_options = dict(
    plot_bgcolor="white",
    margin=dict(
        pad=10,
        autoexpand=True,
        l=margin, r=margin, t=1.5*margin, b=margin
    ),
    width=900,
    height=500,
)

In [8]:
def human_format(num, round_to=2):
    magnitude = 0
    while abs(num) >= 1000:
        magnitude += 1
        num = round(num / 1000.0, round_to)
    return '{:.{}f}{}'.format(num, round_to, ['', 'K', 'M', 'G', 'T', 'P'][magnitude])

In [9]:
def plot_bars(
    data, config,
    title=None, fontsize=14, font_family="Helvetica", round_to=1,
    ylabel=None, log=False,
):
    bars = []
    
    def sort_by_bench_size(key):
        # print(key)
        # print(bench)
        # print(data.iloc[key.index]["Value"])
        values = data.iloc[key.index]["Value"]
        return values
    
    data = data.sort_values(
        by=["Simulator", "Benchmark"], key=sort_by_bench_size)
    
    data = data.set_index(["Simulator"])
    
    data["ValueStr"] = data["Value"].apply(lambda v: human_format(v, round_to=round_to))
    simulators = data.index.get_level_values("Simulator").unique().tolist()
    benchmarks = data["Benchmark"].unique().tolist()
    
    for i, sim in enumerate(simulators):
        # add hardware GPU name
        name = sim
        if sim == "Hardware":
            name += f" ({config.name})"
        
        sim_data = data.loc[[sim]]
        
        bars.append(go.Bar(
            x = sim_data["Benchmark"],
            y = sim_data["Value"],
            text = sim_data["ValueStr"],
            textposition='auto',
            textangle=0,
            textfont = dict(
                size=fontsize - 2,
                color="black",
            ),
            hovertemplate = (
                "<b>%{x}</b><br>" +
                "%{y:.2f}<br>"
            ),
            name=str(name),
            marker=dict(
                color=sim_color[sim],
                line=dict(
                    color='rgba(50, 171, 96, 1.0)',
                    width=0
                ),
            ),
        ))

    layout = go.Layout(
        font_family=font_family,
        font_color="black",
        font_size=fontsize,
        title=dict(
            text=title,
            x=0.5,
            y=0.95,
            xanchor="center",
            yanchor="top",
        ),
        yaxis=go.layout.YAxis(
            title=ylabel,
            gridcolor="gray",
            zerolinecolor="gray",
        ),
        xaxis=go.layout.XAxis(
            tickfont=dict(
                size=0.8 * fontsize,
            ),
            dividerwidth=0,
            dividercolor="white",
        ),
        hoverlabel=dict(
            bgcolor="white",
            font_size=fontsize,
            font_family=font_family,
        ),
        barmode="group",
        bargroupgap=0.1,
        bargap=0.25,
        showlegend=True,
        **default_layout_options,
    )
    if log:
        layout.yaxis.type = "log"
    fig = go.Figure(data=bars, layout=layout)
    return fig

In [10]:
def plot_bars_exec_time(
    data, config,
    title=None, fontsize=14, font_family="Helvetica", round_to=1,
    ylabel=None, log=False,
):
    bars = []
    def sort_by_bench_size(key):
        # print(key)
        # print(bench)
        # print(data.iloc[key.index]["Value"])
        values = data.iloc[key.index]["Value"]
        return values
    
    data = data.sort_values(
        by=["Simulator", "Benchmark"], key=sort_by_bench_size)
    
    data = data.set_index(["Simulator", "Kind"])
    # data = data.set_index(["Simulator", "Kind"])
    # data = data.sort_values(by="Benchmark")
    # data = data.sort_index()
    simulators = data.index.get_level_values("Simulator").unique().tolist()
    # data.index.get_level_values("Simulator")
    # simulators = [s for s in simulators if ["Value"]]
    
    for i, sim in enumerate(simulators):
        # add hardware GPU name
        name = sim
        if sim == "Hardware":
            name += f" ({config.name})"
        
        # add bars
        trace_time = data.loc[data.index == (sim, "Trace")]
        sim_time = data.loc[data.index == (sim, "Sim")]
        total_time = trace_time["Value"].values +  sim_time["Value"].values
        if total_time.sum() == 0:
            # skip simulator
            continue
        
        benchmarks = trace_time["Benchmark"].values
        x = [benchmarks, [abbr[sim]] * len(benchmarks)]
        # flat_x = [item for sublist in x for item in sublist]
        # y = [10] * len(flat_x)
        y = trace_time["Value"]
        if sim != "Hardware":
            y = y.round(round_to)
        
        bars.append(go.Bar(
            x = x,
            y = y,
            text = y.apply(lambda v: f"tracing<br>{v}"),
            textposition='auto',
            textangle=0,
            showlegend=False,
            textfont = dict(
                size=fontsize - 2,
                color="black",
            ),
            hovertemplate = (
                "<b>%{x}</b><br>" +
                "%{y:.2f}<br>"
            ),
            name=str(name),
            marker=dict(
                color=sim_color[sim],
                line=dict(
                    color="white",
                    width=2,
                ),
            ),
        ))
        
        # continue
        if sim != "Hardware":
            total_time = total_time.round(round_to)
        # print(total_time)
        bars.append(go.Bar(
            x = x,
            y = total_time,
            text = total_time,
            textposition='auto',
            textangle=0,
            textfont = dict(
                size=fontsize - 2,
                color="black",
            ),
            hovertemplate = (
                "<b>%{x}</b><br>" +
                "%{y:.2f}<br>"
            ),
            name=str(name),
            marker=dict(
                color=sim_color[sim],
                line=dict(
                    color="white",
                    width=2,
                ),
            ),
        ))
    
    # add empty separator
    x = [benchmarks, [""] * len(benchmarks)]
    y = [0] * len([item for sublist in x for item in sublist])
    bars.append(go.Bar(
        x = x,
        y = y,
        showlegend=False,
    ))
    
    margin = 50
    layout = go.Layout(
        font_family=font_family,
        font_color="black",
        font_size=fontsize,
        title=dict(
            text=title,
            x=0.5,
            y=0.95,
            xanchor="center",
            yanchor="top",
        ),
        yaxis=go.layout.YAxis(
            title=ylabel,
            gridcolor="gray",
            zerolinecolor="gray",
            # range=[min_axis_val * 0.9 ,max_axis_val*1.1]
        ),
        xaxis=go.layout.XAxis(
            # title="Benchmark",
            tickfont=dict(
                size=0.8 * fontsize,
            ),
            dividerwidth=0,
            dividercolor="white",
        ),
        hoverlabel=dict(
            bgcolor="white",
            font_size=fontsize,
            font_family=font_family,
        ),
        barmode="stack",
        # bargroupgap=0.1,
        bargap=0.1,
        showlegend=True,
        **default_layout_options,
    )
    if log:
        layout.yaxis.type = "log"
    return go.Figure(data=bars, layout=layout)

In [11]:
def plot_mean_slowdown(df):
    slowdown_df = df
    # print(slowdown_df.shape)
    slowdown_df = slowdown_df.drop(columns=["Kind"])
    slowdown_df = slowdown_df.groupby([
        "Simulator", "Benchmark", "Config"]).sum().reset_index()
    slowdown_df = slowdown_df[slowdown_df["Value"] > 0]

    hw = slowdown_df[slowdown_df["Simulator"] == "Hardware"]
    hw = hw[["Benchmark", "Config", "Value"]]
    hw = hw.rename(columns={"Value": "HardwareValue"})

    final_slowdown_df = []

    # slowdown_df = slowdown_df[slowdown_df["Value"] > 0]
    # print(slowdown_df.shape)
    # [["Simulator", "Value"]]
    # slowdown_df_group = slowdown_df.groupby("Simulator")
    for sim, sim_df in slowdown_df.groupby("Simulator"):
        if sim == "Hardware":
            continue
        # print(sim)
        # sim_df = sim_df[sim_df["Value"] > 0]
        # get all the benchmarks
        # Value_x	Value_y

        # hw = hw[["Benchmark", "Config", "Value"]]
        # hw = hw.groupby(["Benchmark", "Config"]).sum()
        # hw = hw["Benchmark", "Config", "Value"]
        # hw = hw.reset_index()
        # print(hw.shape)

        # print(sim_df.shape)
        # sim_df = sim_df[["Benchmark", "Config", "Value"]]
        sim_df = sim_df.rename(columns={"Value": "SimValue"})
        sim_df = sim_df.merge(hw, how="inner", on=["Benchmark", "Config"])
        sim_df["Value"] = sim_df["SimValue"] / sim_df["HardwareValue"]
        # [["Config", "Slowdown"]]
        # [["Config", "Slowdown"]]
        sim_df = sim_df[["Simulator", "Config", "Value"]]
        sim_df = sim_df.groupby(["Simulator", "Config"]).mean()
        sim_df = sim_df.reset_index()
        # ["Slowdown"]
        # print(sim_df.shape)
        final_slowdown_df.append(sim_df)
        # join based on the benchmarks
        # break
    # hw
    final_slowdown_df = pd.concat(final_slowdown_df)
    final_slowdown_df = final_slowdown_df.rename(columns={"Config": "Benchmark"})
    return final_slowdown_df, plot_bars(
        data=final_slowdown_df,
        config=config,
        ylabel="Slowdown factor",
        title=f"Mean simulation slowdown",
        log=True,
    )

# final_slowdown_df, fig = plot_mean_slowdown(all_benches_metric_df)
# fig.show()
# final_slowdown_df

In [12]:
# => per config, benchmark and input, plot bars for each simulator

metrics = {
    gpusims.plot.metrics.ExecutionTime: plot_bars_exec_time,
    gpusims.plot.metrics.Cycles: plot_bars,
    gpusims.plot.metrics.L2ReadHit: plot_bars,
    gpusims.plot.metrics.DRAMReads: plot_bars,
    gpusims.plot.metrics.DRAMWrites: plot_bars,
    gpusims.plot.metrics.IPC: plot_bars,
    gpusims.plot.metrics.InstructionCount: plot_bars,
    gpusims.plot.metrics.L2Writes: plot_bars,
}
if False:
    metrics = {
        # gpusims.plot.metrics.L2Writes: plot_bars,
        gpusims.plot.metrics.ExecutionTime: plot_bars_exec_time,
        # gpusims.plot.metrics.Cycles: plot_bars,
        # gpusims.plot.metrics.IPC: plot_bars,
        # gpusims.plot.metrics.InstructionCount: plot_bars,
    }

for metric_cls, metrics_plot_func in metrics.items():
    print(metric_cls.name)
    ylabel = metric_cls.name
    if metric_cls.unit is not None:
        ylabel += f" [{metric_cls.unit}]"

    all_benches_metric_df = []
    for config_name, config in selected_configs.items():
        for bench_name, bench in selected_benchmarks.items():
            # print(config_name, bench_name)
            all_metric_df = []
            for inp in bench.inputs:
                plot_data = PlotData(benchmark=bench, config=config, inp=inp)
                for (sim_name, sim) in selected_simulators.items():
                    if not bench.enabled(sim_name):
                        continue
                    if not inp.enabled(sim_name):
                        continue
                    # print(sim_name, config_name, bench_name)
                    bench_config = sim(
                        run_dir=run_dir / sim_name.lower(),
                        benchmark=bench,
                        config=config,
                    )
                    if not bench_config.input_path(inp).is_dir():
                        print(f"WARN: {bench_config.input_path(inp)} does not exist")
                        continue

                    plot_data[sim_name] = bench_config.load_dataframe(inp)

                metric = metric_cls(plot_data)
                metric_df = metric.compute()
                metric_df["Benchmark"] = f"{bench.name}<br>{inp.args}"
                metric_df["Config"] = config.name

                all_metric_df.append(metric_df)
                all_benches_metric_df.append(metric_df)
                # continue

                # print(metric_df)
                fig = metrics_plot_func(
                    data=metric_df,
                    config=config,
                    ylabel=ylabel,
                    title=f"{metric_cls.name} for {bench.name} {inp.args} ({config.name})",
                    log=metric_cls.log,
                )
                filename = ["bar", metric.name, bench.name, config.key, inp.sanitized_name()]
                filename = Path("./figs") / gpusims.utils.slugify("_".join(filename))
                filename = filename.with_suffix(".pdf")
                filename.parent.mkdir(parents=True, exist_ok=True)
                fig.write_image(filename, format='pdf')
                # print("wrote", filename)

            all_metric_df = pd.concat(all_metric_df)
            fig = metrics_plot_func(
                data=all_metric_df,
                config=config,
                ylabel=ylabel,
                title=f"{metric_cls.name} for {bench.name} ({config.name})",
                log=metric_cls.log,
            )
            filename = ["all_inputs_bar", metric_cls.name, bench.name, config.key]
            filename = Path("./figs") / gpusims.utils.slugify("_".join(filename))
            filename = filename.with_suffix(".pdf")
            filename.parent.mkdir(parents=True, exist_ok=True)
            fig.write_image(filename, format='pdf')
            # print("wrote", filename)
            # break

    if metric_cls is gpusims.plot.metrics.ExecutionTime:
        all_benches_metric_df = pd.concat(all_benches_metric_df)
        _, fig = plot_mean_slowdown(all_benches_metric_df)
        filename = Path("./figs") / "mean_slowdown.pdf"
        filename.parent.mkdir(parents=True, exist_ok=True)
        fig.write_image(filename, format='pdf')

# all_metric_df

Execution Time
Cycles
Total L2 Read Hits
Total DRAM Reads
Total DRAM Writes
Total IPC
Total Instruction Count
Total L2 Writes


In [13]:
# debug dataframe
# sim = gpusims.MULTI2SIM
# sim = gpusims.MACSIM
# sim = gpusims.TEJAS
# sim = gpusims.ACCELSIM_SASS
sim = gpusims.ACCELSIM_PTX
# sim = gpusims.NATIVE
# benchmark = benchmarks["babelstream"]
# benchmark = benchmarks["cuda4-matrixmul"]
benchmark = benchmarks["cuda6-transpose"]
benchmark = benchmarks["vectoradd"]
# config=configs["sm86_rtx3070"]
config=configs["sm6_gtx1080"]
# config=configs["sm86_a4000"]
bench_config = gpusims.SIMULATORS[sim](
    run_dir=run_dir / sim.lower(),
    benchmark=benchmark,
    config=config,
)
assert bench_config.benchmark.enabled(sim)
bench_input = bench_config.benchmark.inputs[-1]
print(sim, benchmark.name, bench_input.args)
df = bench_config.load_dataframe(bench_input)
df = df.round(0)
# pprint(df.columns.tolist())
# print(df.index)
print(df.shape)
# df.T[~df.T.index.str.contains(r"^Config.|SM \d+", re.IGNORECASE)] # multi2sim
# df.T[~df.T.index.str.contains(r"_CORE_\d+$", re.IGNORECASE)] # macsim
# df.T[df.T.index.str.contains(r"tex_op_read.sum", re.IGNORECASE)] # filtering
df.T[df.T.index.str.contains(r"l2_cache_read_hit", re.IGNORECASE)] # filtering
# df.T
# df.T

accelsim-ptx vectorAdd 1000000
(1, 26)


,0
stat,
l2_cache_read_hit,0.0
